# Import Model

In [1]:
from cameo import models
bigg_list = models.index_models_bigg()

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
# Find GEMS that starts with "Eschericia coli" and ends with "MG1655"
e_coli_mg1655 = bigg_list[bigg_list.organism.str.contains("^Escherichia coli.*MG1655$")]
e_coli_mg1655

,bigg_id,gene_count,reaction_count,organism,metabolite_count
0,e_coli_core,137,95,Escherichia coli str. K-12 substr. MG1655,72
2,iAF1260,1261,2382,Escherichia coli str. K-12 substr. MG1655,1668
3,iAF1260b,1261,2388,Escherichia coli str. K-12 substr. MG1655,1668
71,iJO1366,1367,2583,Escherichia coli str. K-12 substr. MG1655,1805
72,iJR904,904,1075,Escherichia coli str. K-12 substr. MG1655,761
76,iML1515,1516,2712,Escherichia coli str. K-12 substr. MG1655,1877


In [3]:
model_id = e_coli_mg1655.iloc[e_coli_mg1655.reaction_count.argmax()].bigg_id
model_id

'iML1515'

# Simulate Model

In [4]:
from cameo import load_model
model = load_model(model_id)
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f3648ea5750
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
model.solver = 'gurobi'

In [6]:
from cameo import fba
%time fba_result = fba(model)

CPU times: user 312 ms, sys: 15.6 ms, total: 328 ms
Wall time: 364 ms


In [7]:
#fba_result.display_on_map("iJO1366.Central metabolism") #seems cameo not updated with escher yet

In [8]:
fba_result[model.reactions.BIOMASS_Ec_iML1515_core_75p37M]

0.8769972144269728

In [9]:
from escher import Builder

In [10]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=fba_result.fluxes,
)
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': -0.022947509112696166, 'SHK3Dr': 0.3…

# Acetic Acid Producer Model
## Play Around, create list of knockouts

In [11]:
# copy model to create mutant
mutant1 = model.copy()

Read LP format model from file /tmp/tmpoauggf0o.lp
Reading time = 0.03 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /tmp/tmp3i55d7pn.lp
Reading time = 0.03 seconds
: 1877 rows, 5424 columns, 21150 nonzeros


In [12]:
import pandas as pd
import numpy as np
# create dummy list of genes to KO
ko_list = pd.DataFrame(np.array(["ACKr", "ATPS4r", "ATPM", "FUM"]),
                   columns=['KO_GENES'])
ko_list

,KO_GENES
0,ACKr
1,ATPS4r
2,ATPM
3,FUM


In [13]:
# Find ID of the corresponding reactions to a gene
ko_id = []
for i in ko_list.KO_GENES:
    #print(i)
    for counter, value in enumerate(mutant1.reactions):
        if i in value.id:
            #print(counter, value.id)
            target = [counter, value.id]
            #print(target)
            ko_id.append(target)
ko_id

[[80, 'ACKr'],
 [798, 'ATPS4rpp'],
 [1434, 'ATPM'],
 [195, 'FUM'],
 [671, 'SUCFUMtpp'],
 [875, 'FUMt2_2pp'],
 [876, 'FUMt2_3pp'],
 [939, 'FUMtex'],
 [2661, 'FUMt1pp']]

In [14]:
# make sure it is the right reaction
for i in ko_id:
    mutant1.reactions[i[0]].knock_out()
    print(mutant1.reactions[i[0]])

ACKr: ac_c + atp_c --> actp_c + adp_c
ATPS4rpp: adp_c + 4.0 h_p + pi_c --> atp_c + h2o_c + 3.0 h_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
FUM: fum_c + h2o_c --> mal__L_c
SUCFUMtpp: fum_p + succ_c --> fum_c + succ_p
FUMt2_2pp: fum_p + 2.0 h_p --> fum_c + 2.0 h_c
FUMt2_3pp: fum_p + 3.0 h_p --> fum_c + 3.0 h_c
FUMtex: fum_e --> fum_p
FUMt1pp: fum_p --> fum_c


In [15]:
# create knockouts
for i in ko_id:
    mutant1.reactions[i[0]].knock_out()
    print(model.reactions[i[0]])
    print("model: "+str(model.reactions[i[0]].bounds))
    print("mutant: "+str(mutant1.reactions[i[0]].bounds))

ACKr: ac_c + atp_c <=> actp_c + adp_c
model: (-1000.0, 1000.0)
mutant: (0, 0)
ATPS4rpp: adp_c + 4.0 h_p + pi_c <=> atp_c + h2o_c + 3.0 h_c
model: (-1000.0, 1000.0)
mutant: (0, 0)
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
model: (6.86, 1000.0)
mutant: (0, 0)
FUM: fum_c + h2o_c <=> mal__L_c
model: (-1000.0, 1000.0)
mutant: (0, 0)
SUCFUMtpp: fum_p + succ_c <=> fum_c + succ_p
model: (-1000.0, 1000.0)
mutant: (0, 0)
FUMt2_2pp: fum_p + 2.0 h_p --> fum_c + 2.0 h_c
model: (0.0, 1000.0)
mutant: (0, 0)
FUMt2_3pp: fum_p + 3.0 h_p --> fum_c + 3.0 h_c
model: (0.0, 1000.0)
mutant: (0, 0)
FUMtex: fum_e <=> fum_p
model: (-1000.0, 1000.0)
mutant: (0, 0)
FUMt1pp: fum_p <=> fum_c
model: (-1000.0, 1000.0)
mutant: (0, 0)


In [16]:
%time fba_knockout_result = fba(mutant1)
fba_knockout_result[mutant1.reactions.BIOMASS_Ec_iML1515_core_75p37M]

CPU times: user 250 ms, sys: 15.6 ms, total: 266 ms
Wall time: 300 ms


0.19672476125372415

In [17]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=fba_knockout_result.fluxes,
)
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0053149128747918654, 'SHK3Dr': 0.0…

In [18]:
mutant1.medium

{'EX_pi_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

## Open List of Knocknout Genes

# Data from Yixin
## Xylitol strain
### Knock outs
* focA, https://www.uniprot.org/uniprot/P0AC23
* pflB, https://www.uniprot.org/uniprot/P09373
* ldhA, https://www.uniprot.org/uniprot/P52643
* adhE, https://www.uniprot.org/uniprot/P0A9Q7
* xylAB, https://www.uniprot.org/uniprot/D1MDQ9
* frdA, https://www.uniprot.org/uniprot/P00363
* cyoB, https://www.uniprot.org/uniprot/P0ABI8
* appB, https://www.uniprot.org/uniprot/P26458

### Missing reaction(s)
* xylose + NADPH -> xylitol + NADP+ https://www.brenda-enzymes.org/enzyme.php?ecno=1.1.1.307

## Isobutyric acid strain
### Knock outs
* focA, https://www.uniprot.org/uniprot/P0AC23
* pflB, https://www.uniprot.org/uniprot/P09373
* aceEF, https://www.uniprot.org/uniprot/P0ACL9
* poxB, https://www.uniprot.org/uniprot/P07003
* tdcE, https://www.uniprot.org/uniprot/P42632
* pflDC, 
* deoC, https://www.uniprot.org/uniprot/P0A6L0
* ydbK, https://www.uniprot.org/uniprot/P52647
* yqhD, https://www.uniprot.org/uniprot/Q46856
* araBA
* xylAB, https://www.uniprot.org/uniprot/D1MDQ9


### Missing reactions(s): 
* isobutyraldehyde to isobutyric acid (forgot the enzyme/gene name)

In [19]:
from bioservices import UniProt
import io
service = UniProt()

In [20]:
df_xylitol = pd.read_csv('data/data_ko_xylitol_strain.txt', header = None,
            skiprows = 1,
            sep = ',',
            names = [
                'gene',
                'uniprot', 
                'strain'])
df_xylitol['strain'] = 'xylytol_mutant'
df_xylitol

,gene,uniprot,strain
0,pflB,https://www.uniprot.org/uniprot/P09373,xylytol_mutant
1,ldhA,https://www.uniprot.org/uniprot/P52643,xylytol_mutant
2,adhE,https://www.uniprot.org/uniprot/P0A9Q7,xylytol_mutant
3,xylAB,https://www.uniprot.org/uniprot/D1MDQ9,xylytol_mutant
4,frdA,https://www.uniprot.org/uniprot/P00363,xylytol_mutant
5,cyoB,https://www.uniprot.org/uniprot/P0ABI8,xylytol_mutant
6,appB,https://www.uniprot.org/uniprot/P26458,xylytol_mutant


In [21]:
df = pd.DataFrame(columns=["Entry", "Entry name", "Gene names  (primary )", "Gene names  (synonym )", "Protein names","Gene ontology (molecular function)","Catalytic activity"])
for i in df_xylitol.uniprot:
    uniprot_id = i.split('/')[-1]
    # Make a query string
    query = "accession:"+uniprot_id
    # Define a list of columns we want to retrive
    columnlist = "id,entry name,genes(PREFERRED),genes(ALTERNATIVE),protein names,go(molecular function),comment(CATALYTIC ACTIVITY)"
    # Run the remote search
    result = service.search(query, frmt="tab", columns=columnlist)
    df_add = (pd.read_table(io.StringIO(result)))
    df = pd.concat([df, df_add])
df = df.reset_index(drop=True)
df_xylitol = pd.merge(df_xylitol, df, left_index=True, right_index=True)
df_xylitol

,gene,uniprot,strain,Entry,Entry name,Gene names (primary ),Gene names (synonym ),Protein names,Gene ontology (molecular function),Catalytic activity
0,pflB,https://www.uniprot.org/uniprot/P09373,xylytol_mutant,P09373,PFLB_ECOLI,pflB,pfl,Formate acetyltransferase 1 (EC 2.3.1.54) (Pyr...,formate C-acetyltransferase activity [GO:0008861],CATALYTIC ACTIVITY: Reaction=acetyl-CoA + form...
1,ldhA,https://www.uniprot.org/uniprot/P52643,xylytol_mutant,P52643,LDHD_ECOLI,ldhA,hslI htpH,D-lactate dehydrogenase (D-LDH) (EC 1.1.1.28) ...,D-lactate dehydrogenase activity [GO:0008720];...,CATALYTIC ACTIVITY: Reaction=(R)-lactate + NAD...
2,adhE,https://www.uniprot.org/uniprot/P0A9Q7,xylytol_mutant,P0A9Q7,ADHE_ECOLI,adhE,ana,Aldehyde-alcohol dehydrogenase [Includes: Alco...,acetaldehyde dehydrogenase (acetylating) activ...,CATALYTIC ACTIVITY: Reaction=a primary alcohol...
3,xylAB,https://www.uniprot.org/uniprot/D1MDQ9,xylytol_mutant,D1MDQ9,D1MDQ9_ECOLX,xylAB,xylB,Xylulose kinase (Xylulokinase) (EC 2.7.1.17) (...,ATP binding [GO:0005524]; xylulokinase activit...,CATALYTIC ACTIVITY: Reaction=ATP + D-xylulose ...
4,frdA,https://www.uniprot.org/uniprot/P00363,xylytol_mutant,P00363,FRDA_ECOLI,frdA,NaN,Fumarate reductase flavoprotein subunit (EC 1....,electron transfer activity [GO:0009055]; FAD b...,CATALYTIC ACTIVITY: Reaction=a quinone + succi...
5,cyoB,https://www.uniprot.org/uniprot/P0ABI8,xylytol_mutant,P0ABI8,CYOB_ECOLI,cyoB,NaN,Cytochrome bo(3) ubiquinol oxidase subunit 1 (...,copper ion binding [GO:0005507]; cytochrome bo...,CATALYTIC ACTIVITY: Reaction=2 a ubiquinol + n...
6,appB,https://www.uniprot.org/uniprot/P26458,xylytol_mutant,P26458,APPB_ECOLI,appB,cbdB cyxB,Cytochrome bd-II ubiquinol oxidase subunit 2 (...,electron transfer activity [GO:0009055]; metal...,CATALYTIC ACTIVITY: Reaction=2 a ubiquinol + n...


In [22]:
df_xylitol.to_csv('data/df_xylitol_knockouts.csv')

In [23]:
df_isobutyric = pd.read_csv('data/data_ko_isobutyric-acid_strain.txt', header = None,
            skiprows = 1,
            sep = ',',
            names = [
                'gene',
                'uniprot', 
                'strain'])
df_isobutyric['strain'] = 'isobutyric_mutant'
df_isobutyric

,gene,uniprot,strain
0,pflB,https://www.uniprot.org/uniprot/P09373,isobutyric_mutant
1,aceEF,https://www.uniprot.org/uniprot/P0ACL9,isobutyric_mutant
2,poxB,https://www.uniprot.org/uniprot/P07003,isobutyric_mutant
3,tdcE,https://www.uniprot.org/uniprot/P42632,isobutyric_mutant
4,pflDC,NaN,isobutyric_mutant
5,deoC,https://www.uniprot.org/uniprot/P0A6L0,isobutyric_mutant
6,ydbK,https://www.uniprot.org/uniprot/P52647,isobutyric_mutant
7,yqhD,https://www.uniprot.org/uniprot/Q46856,isobutyric_mutant
8,araBA,NaN,isobutyric_mutant
9,xylAB,https://www.uniprot.org/uniprot/D1MDQ9,isobutyric_mutant


In [24]:
df_isobutyric = df_isobutyric.dropna().reset_index(drop=True)
df_isobutyric

,gene,uniprot,strain
0,pflB,https://www.uniprot.org/uniprot/P09373,isobutyric_mutant
1,aceEF,https://www.uniprot.org/uniprot/P0ACL9,isobutyric_mutant
2,poxB,https://www.uniprot.org/uniprot/P07003,isobutyric_mutant
3,tdcE,https://www.uniprot.org/uniprot/P42632,isobutyric_mutant
4,deoC,https://www.uniprot.org/uniprot/P0A6L0,isobutyric_mutant
5,ydbK,https://www.uniprot.org/uniprot/P52647,isobutyric_mutant
6,yqhD,https://www.uniprot.org/uniprot/Q46856,isobutyric_mutant
7,xylAB,https://www.uniprot.org/uniprot/D1MDQ9,isobutyric_mutant


In [25]:
df = pd.DataFrame(columns=["Entry", "Entry name", "Gene names  (primary )", "Gene names  (synonym )", "Protein names","Gene ontology (molecular function)","Catalytic activity"])
for i in df_isobutyric.uniprot:
    uniprot_id = i.split('/')[-1]
    # Make a query string
    query = "accession:"+uniprot_id
    # Define a list of columns we want to retrive
    columnlist = "id,entry name,genes(PREFERRED),genes(ALTERNATIVE),protein names,go(molecular function),comment(CATALYTIC ACTIVITY)"
    # Run the remote search
    result = service.search(query, frmt="tab", columns=columnlist)
    df_add = (pd.read_table(io.StringIO(result)))
    df = pd.concat([df, df_add])
df = df.reset_index(drop=True)
df_isobutyric = pd.merge(df_isobutyric, df, left_index=True, right_index=True)
df_isobutyric

,gene,uniprot,strain,Entry,Entry name,Gene names (primary ),Gene names (synonym ),Protein names,Gene ontology (molecular function),Catalytic activity
0,pflB,https://www.uniprot.org/uniprot/P09373,isobutyric_mutant,P09373,PFLB_ECOLI,pflB,pfl,Formate acetyltransferase 1 (EC 2.3.1.54) (Pyr...,formate C-acetyltransferase activity [GO:0008861],CATALYTIC ACTIVITY: Reaction=acetyl-CoA + form...
1,aceEF,https://www.uniprot.org/uniprot/P0ACL9,isobutyric_mutant,P0ACL9,PDHR_ECOLI,pdhR,aceC genA yacB,Pyruvate dehydrogenase complex repressor,DNA binding [GO:0003677]; DNA-binding transcri...,NaN
2,poxB,https://www.uniprot.org/uniprot/P07003,isobutyric_mutant,P07003,POXB_ECOLI,poxB,NaN,Pyruvate dehydrogenase [ubiquinone] (EC 1.2.5....,flavin adenine dinucleotide binding [GO:005066...,CATALYTIC ACTIVITY: Reaction=a ubiquinone + H2...
3,tdcE,https://www.uniprot.org/uniprot/P42632,isobutyric_mutant,P42632,TDCE_ECOLI,tdcE,yhaS,PFL-like enzyme TdcE (Keto-acid formate acetyl...,2-ketobutyrate formate-lyase activity [GO:0043...,CATALYTIC ACTIVITY: Reaction=2-oxobutanoate + ...
4,deoC,https://www.uniprot.org/uniprot/P0A6L0,isobutyric_mutant,P0A6L0,DEOC_ECOLI,deoC,dra thyR,Deoxyribose-phosphate aldolase (DERA) (EC 4.1....,deoxyribose-phosphate aldolase activity [GO:00...,CATALYTIC ACTIVITY: Reaction=2-deoxy-D-ribose ...
5,ydbK,https://www.uniprot.org/uniprot/P52647,isobutyric_mutant,P52647,NIFJ_ECOLI,ydbK,NaN,Probable pyruvate-flavodoxin oxidoreductase (E...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",CATALYTIC ACTIVITY: Reaction=CoA + 2 H(+) + ox...
6,yqhD,https://www.uniprot.org/uniprot/Q46856,isobutyric_mutant,Q46856,YQHD_ECOLI,yqhD,NaN,Alcohol dehydrogenase YqhD (EC 1.1.1.-),alcohol dehydrogenase (NADP+) activity [GO:000...,CATALYTIC ACTIVITY: Reaction=a primary alcohol...
7,xylAB,https://www.uniprot.org/uniprot/D1MDQ9,isobutyric_mutant,D1MDQ9,D1MDQ9_ECOLX,xylAB,xylB,Xylulose kinase (Xylulokinase) (EC 2.7.1.17) (...,ATP binding [GO:0005524]; xylulokinase activit...,CATALYTIC ACTIVITY: Reaction=ATP + D-xylulose ...


In [26]:
df_isobutyric.to_csv('data/df_isobutyric_knockouts.csv')

In [27]:
ko_id = []
for i in ko_list.KO_GENES:
    #print(i)
    for counter, value in enumerate(model.genes):
        if i in value.name:
            #print(counter, value.id)
            target = [counter, value.id]
            #print(target)
            ko_id.append(target)
ko_id

[]

In [28]:
for i in model.genes:
    print(i.name)

glyA
ltaE
cysG
hemF
talB
gor
tktB
pyrD
glcF
garL
glcE
glcD
pfkA
fabF
lpxL
lpxM
trpA
lpxP
nadC
acpP
aas
dapE
tktA
trpB
glpD
glpB
glpC
glpA
tyrB
ilvE
aroE
ydiB
proA
ynfH
manA
ribD
dmsA
ynfF
ynfE
dmsC
ynfG
dmsB
ribB
ribF
ghrB
nagB
ybjI
cof
adhP
cobC
yigB
ribC
yrfG
ispA
hemD
apt
hemE
deoA
ispB
nanK
nanA
can
ahr
cynT
katG
purH
katE
ybbO
guaB
fadB
aldB
entF
fadJ
wecG
gltX
yghA
thiE
yqhD
dgoK
udk
bglA
gpt
ascB
rhaA
hpt
adk
ligA
bglB
ndk
pyrC
malQ
nudC
pyrF
nadR
nadD
gltA
paaK
icd
apaH
thiD
adhE
pdxK
mhpF
udp
wecA
wecF
acs
hldE
ppx
deoD
ppa
patA
otsA
umpG
ybjG
pck
pgpB
tnaA
metC
maeA
dadX
yibQ
bacA
alr
alaC
treF
alaA
cmk
nudJ
xylB
pyrE
araB
ribE
gmk
rbsK
atoD
ilvD
glpK
yfbT
atoA
deoB
aspC
ydiF
yihX
yggF
grcA
fbp
tdcE
rfbA
yidA
rffH
rfbB
glpX
rffG
pflC
glgC
glf
rfbC
pykA
rfbD
pflA
pykF
pflB
allB
srlQ
pflD
kdsD
frdC
murB
leuD
frdB
frdA
leuC
frdD
ilvI
poxB
ilvH
ilvB
pta
ilvN
aroA
trpD
eutD
tdcD
trpE
pheA
purT
ackA
tyrA
aroC
waaA
trpC
eno
ispU
fbaA
fbaB
lpxH
mgsA
lpxB
ppsA
pgi
lpxK
pgk
mazG
eda
mu

In [29]:
ko_id = []
for i in ko_list_xylitol.KO_GENES:
    print(i)
    for counter, value in enumerate(model.reactions):
        if i in value.id:
            #print(counter, value.id)
            target = [counter, value.id]
            #print(target)
            ko_id.append(target)
ko_id

NameError: name 'ko_list_xylitol' is not defined

## Create Knockouts - Xylitol 

## Simulate Knockouts

In [ ]:
# Create Knockout - Acetic acid auxotroph


In [ ]:
# Test the Knockout - auxotroph


In [ ]:
# Test dFBA


In [ ]:
# Create coculture model